<a href="https://colab.research.google.com/github/Levis0045/SCIA-CRF_LF/blob/0.1/training/experimentations_crf_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading data from Masakhane folder

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip3 install pandas python_crfsuite summarytools sklearn_crfsuite unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.3/784.3 KB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.2/385.2 KB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 KB 39.3 MB/s eta 0:00:00
  Created wheel for summarytools: filename

In [2]:
!pip3 install iteration_utilities matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.6/344.6 KB 10.4 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/masakhane-io/masakhane-ner.git

Cloning into 'masakhane-ner'...
remote: Enumerating objects: 3923, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 3923 (delta 78), reused 103 (delta 50), pack-reused 3734
Receiving objects: 100% (3923/3923), 54.21 MiB | 20.28 MiB/s, done.
Resolving deltas: 100% (1666/1666), done.
Updating files: 100% (2133/2133), done.


In [4]:
from pathlib import Path
import pandas as pd

- normalization des données d'entrainement et application de quelques corrections
  - ajout de point à la fin des phrases qui n'en ont pas 
  - considérer les tirets comme partie de "I-LOC" ?
  - ŋkaʼ (6) est à la fois O et B-DATE+chiffre (37)
  - non standardisation de l'orthographe: 
        Afrika I-ORG  / Afríkà B-LOC / Afrikǎ I-LOC / Afrika B-LOC / Afrika I-ORG
        / afika B-LOC / Afríka B-LOC / Africa I-ORG / Afrika O / Afríkâ B-LOC

      Nəmo B-ORG / NƏMO O / Nə̀mò B-ORG

- construire une stratégie d'augmentation sur les entités existantes
  - position to position augmentation
  - 


- construire un classifier avec 

In [5]:
# Reading folder path
# Context Masahkane community
# Ramshaw and Marcus (1995) = IOB data format

bbj_ner_path = Path('./masakhane-ner/MasakhaNER2.0/data/bbj')
dev_data_path   = bbj_ner_path / 'dev.txt'
train_data_path = bbj_ner_path / 'train.txt'
test_data_path  = bbj_ner_path / 'test.txt'



In [6]:
from auglib import augment_ner_iob_data
from auglib import read_format_iob_data, list_to_pd_format

In [7]:
# read and extract iob data
extracted_train_data, pd_train_data, only_train_ner_data, o_train_ner_data = read_format_iob_data(train_data_path)
extracted_test_data, pd_test_data, only_test_ner_data, o_test_ner_data = read_format_iob_data(test_data_path)
extracted_dev_data, pd_dev_data, only_dev_ner_data, o_dev_ner_data = read_format_iob_data(dev_data_path)

# quelques stats liminaires
print("Total number of sentences in the train dataset: {:,}".format(pd_train_data["sentence_id"].nunique()))
print("Total words in the train dataset: {:,}".format(pd_train_data.shape[0]))
print("Total number of sentences in the test dataset: {:,}".format(pd_test_data["sentence_id"].nunique()))
print("Total words in the test dataset: {:,}".format(pd_test_data.shape[0]))
print("Total number of sentences in the dev dataset: {:,}".format(pd_dev_data["sentence_id"].nunique()))
print("Total words in the dev dataset: {:,}".format(pd_dev_data.shape[0]))


Total number of sentences in the train dataset: 3,384
Total words in the train dataset: 50,623
Total number of sentences in the test dataset: 966
Total words in the test dataset: 15,924
Total number of sentences in the dev dataset: 483
Total words in the dev dataset: 7,745


In [8]:


#print(org_list[0:12])
#org_list_group   = extract_iob_group_entities(org_list, output_format='list', remove_duplicates=True)
#date_list_group = extract_iob_group_entities(date_list)
#loc_list_group  = extract_iob_group_entities(loc_list)
#per_list_group  = extract_iob_group_entities(per_list)
#org_list_group[:12]
#print(only_train_ner_data[10])


data_extract = [('Mdyə̂faʼ', 'O'), ('mtəŋláʼ', 'B-LOC'), ('shyə̂ŋkaʼ', 'I-LOC'), (',', 'O'), 
('təŋláʼ', 'B-LOC'), ('ŋkaʼ', 'I-LOC'), ('gə́', 'O'), ('təŋláʼ', 'O'), 
('Adamáwǎ', 'B-LOC'), ('kuʼ', 'O'), ('dəŋ', 'O'), ('é', 'O'), ('.', 'O'), ('.', 'O')]
#detect_iob_tag_position(data_extract)

#results_augment = augment_sentence(sent, list_ent_aug=ents_groups)
#for sent in results_augment: print('\t', " ".join([i[0] for i in sent]))

In [9]:
augmented_train_ner_data = augment_ner_iob_data(only_train_ner_data)

print(f' Train data: {len(only_train_ner_data)} \n Augmented data: {len(augmented_train_ner_data)}')

 Train data: 1771 
 Augmented data: 1219850


In [10]:
pd_augment_ner_iob_data = list_to_pd_format(augmented_train_ner_data)
pd_augment_ner_iob_data

,sentence_id,word,iob_tag
0,1,A,O
1,1,shə́ŋ,O
2,1,gaə́,O
3,1,DG,O
4,1,Cameroon,B-ORG
...,...,...,...
20391456,1906795,pú,O
20391457,1906795,pútə́,O
20391458,1906795,é,O
20391459,1906795,lə,O


# Analyzing data 

In [11]:
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# quantification des tags/phrases dans le corpus d'entrainement
# credit: https://github.com/6chaoran/jupyter-summarytools
from summarytools import dfSummary, tabset

tabset({
    "train_data": dfSummary(pd_augment_ner_iob_data).render(),
    "test_data": dfSummary(pd_test_data).render(),
    "dev_data": dfSummary(pd_dev_data).render()
})

# Un controle visuel des données et ce tableau montre clairement qu'il y a 
# des problématiques de cohérence d'annotations avec l'usage du format IOB.

In [ ]:

pd_augment_ner_iob_data[pd_augment_ner_iob_data.iob_tag != 'O']["iob_tag"]\
.value_counts().plot(kind="bar", figsize=(10,5))

# On remarque ici que le tag date représente 40% des tags (en position I ou B)

In [17]:

word_counts = pd_augment_ner_iob_data.groupby("sentence_id")["word"].agg(["count"])
word_counts = word_counts.rename(columns={"count": "Word count"})
#word_counts.hist(bins=25, figsize=(8,6))

# On constate que le nombre de mots moyens par phrase est de 15
MAX_LENGTH_SENTENCE = word_counts.max()[0]
print("La phrase la plus longue contient {} mots.".format(MAX_LENGTH_SENTENCE))
longest_sentence_id = word_counts[word_counts["Word count"]==MAX_LENGTH_SENTENCE].index[0]
print("ID de la plus longue phrase {}.".format(longest_sentence_id))
longest_sentence = pd_augment_ner_iob_data[pd_augment_ner_iob_data["sentence_id"]==longest_sentence_id]["word"].str.cat(sep=' ')
print(f"\nLa phrase la plus longue du corpus est:\n {longest_sentence}")


La phrase la plus longue contient 59 mots.
ID de la plus longue phrase 118931.

La phrase la plus longue du corpus est:
 Nwə yə pú kə yɔ̌ Fəládɛ̂ dɔ̂lɔ̌ Pásə́kà Tyə́ʼ kwɔ̂ʼ kɛ́bəŋ Yésô Tyə́ʼ fîʼ zhwenyə dɔ̂lɔ̌ Tyə́ʼ kwɔ̂ʼ kɛ́bəŋ Malyâ Krǐsì Lamadâŋ Laʼnwə mjʉ̀jʉ̀ Tá lə Ondo André Marie ( Mle 599 438 - T ) Tâdyə̌ mkətú' bə ě tʉɔthə́ bí pə dzə̌ é nə́ŋ cyə̂ mnə́ hɔ̂ bí nə́ pîŋ pú a fa' yə pǐŋ lə .


In [18]:
all_words = list(set(pd_augment_ner_iob_data["word"].values))
all_tags = list(set(pd_augment_ner_iob_data["iob_tag"].values))

print("Nombre de mots uniques: {}".format(pd_augment_ner_iob_data["word"].nunique()))
print("Nombre de tags uniques : {}".format(pd_augment_ner_iob_data["iob_tag"].nunique()))

Nombre de mots uniques: 6176
Nombre de tags uniques : 9


# Features engineering

In [19]:
import unidecode
import re
from datetime import datetime
import string
import math
import unicodedata

In [20]:
words_caracters = set([y.lower() for x in all_words for y in x])
all_caracters = string.punctuation+string.ascii_letters+string.digits+''
tone_caracters = list(set([x for x in words_caracters if x not in all_caracters]))
cpm_search = re.compile(str(tone_caracters))
print(tone_caracters)

['̂', 'è', 'ǒ', 'œ', 'ɔ', 'ú', '̌', 'û', 'ç', 'ə', 'ǔ', '᷆', 'à', 'ɛ', 'ï', 'ʉ', 'â', 'ʼ', 'ɨ', 'á', 'ó', 'ô', 'ǝ', 'ǐ', 'ê', 'ꞌ', 'ì', 'í', '᷅', 'î', 'é', 'ɓ', 'ŋ', 'ě', 'ɑ', '̣', '°', 'ş', 'ò', 'ù', 'ͻ', 'ë', '̀', '̧', 'ᵾ', 'ǎ', '©', '́']


In [21]:
bantou_tones = [f"{x} " for x in " ́̄̀̌̂" if x != " "]
string_tones = "".join(bantou_tones)
tones_search = re.compile(string_tones)
print([x for x in "ntâmgǒ"])
print(tone_caracters, string_tones)

bantou_letters = string.ascii_letters+"ǝɔᵾɓɨşœɑʉɛɗŋøẅëïə"


def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    #print([x for x in nfkd_form if x not in string.ascii_letters])
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf8')

def extract_tone(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    #print([x for x in nfkd_form if x not in string.ascii_letters])
    tones = [x for x in nfkd_form if x not in bantou_letters]
    return " ".join(tones)

def number_tone_word(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    len_tone_str = len([x for x in nfkd_form if x not in bantou_letters])
    return len_tone_str

def word_decomposition(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    word_decomp = " ".join([x for x in nfkd_form ])
    return word_decomp

def compare_two_words(input_str1, input_str2):
    nfkd_form1 = unicodedata.normalize('NFKD', input_str1)
    nfkd_form2 = unicodedata.normalize('NFKD', input_str2)
    len_tone_str1 = len([x for x in nfkd_form1 if x not in bantou_letters])
    len_tone_str2 = len([x for x in nfkd_form2 if x not in bantou_letters])
    only_ascii = nfkd_form1.encode('ASCII', 'ignore')
    return only_ascii.decode('utf8')

non_tone = remove_accents("fə̀fə̀")
print(len("fə̀fə̀"), len(non_tone), number_tone_word("fə̀fə̀"), 
      "---"+extract_tone("fə̀fə̀"))

['n', 't', 'â', 'm', 'g', 'ǒ']
['̂', 'è', 'ǒ', 'œ', 'ɔ', 'ú', '̌', 'û', 'ç', 'ə', 'ǔ', '᷆', 'à', 'ɛ', 'ï', 'ʉ', 'â', 'ʼ', 'ɨ', 'á', 'ó', 'ô', 'ǝ', 'ǐ', 'ê', 'ꞌ', 'ì', 'í', '᷅', 'î', 'é', 'ɓ', 'ŋ', 'ě', 'ɑ', '̣', '°', 'ş', 'ò', 'ù', 'ͻ', 'ë', '̀', '̧', 'ᵾ', 'ǎ', '©', '́'] ́ ̄ ̀ ̌ ̂ 
6 2 2 ---̀ ̀


In [22]:
# l'ajout des tags suivants au mot courant améliore signficativement le modèle
# l'ajout des informations sur les tons

def word2features(sent, i):
    word = sent[i][0]
    tagword = sent[i][1]
    len_tone = number_tone_word(word)
    len_word = len(word) / 2
    features = {
        'bias': 1.0,
        'word': word,
        'word.tones': extract_tone(word),
        'word.normalized': unicodedata.normalize('NFKD', word),
        #'word.letters': word_decomposition(word),
        'word.position': i,
        #'word[:3]': word[:3], impacte négativement les résultats
        #'word[:2]': word[:2],
        #'word[-3:]': word[-3:],
        #'word[-2:]': word[-2:],
        #'word.middle_start': word[:int(len_word)],
        #'word.middle_end': word[int(len_word):],
        'word.has_hyphen': '-' in word,
        #'word.unaccent': remove_accents(word),
        'word.lower()': word.lower(),
        'word.start_with_capital': word[0].isupper(),
        'word.have_tone': True if len_tone>0 else False,
        #'word.len_tones': len_tone,
        'word.ispunctuation': (word in string.punctuation),
        'word.isdigit()': word.isdigit()
    }
    if word == '.': features['EOS'] = True

    if i > 0:
        word1 = sent[i-1][0]
        tagword1 = sent[i-1][1]
        len_tone1 = number_tone_word(word1)
        len_word1 = len(word1) / 2
        features.update({
            '-1:word': word1,
            '-1:word.position': i-1,
            '-1:word.letters': word_decomposition(word1),
            '-1:word.normalized': unicodedata.normalize('NFKD', word1),
            '-1:word.start_with_capital': word1[0].isupper(),
            '-1:len(word1)': len(word1),
            '-1:word.lower()': word1.lower(),
            #'-1:word.tag()': tagword1,
            #'-1:word.unaccent': remove_accents(word1),
            #'-1:word.middle_start': word1[:int(len_word1)],
            #'-1:word.middle_end': word1[int(len_word1):],
            #'-1:word.have_tone': True if len_tone1>0 else False,
            #'-1:word.len_tones': len_tone1,
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.ispunctuation': (word1 in string.punctuation)
        })
        if tagword not in ['B-ORG','B-LOC']: features.update({'-1:word.tag()': tagword1})
    else: features['BOS'] = True
    
    """
    if i > 1:
        word2 = sent[i-2][0]
        tagword2 = sent[i-2][1]
        len_tone2 = number_tone_word(word2)
        len_word2 = len(word2) / 2
        features.update({
            '-2:word': word2,
            '-2:word.position': i-2,
            '-2:word.letters': word_decomposition(word2),
            '-2:word.normalized': unicodedata.normalize('NFKD', word2),
            '-2:word.start_with_capital': word2[0].isupper(),
            '-2:len(word2)': len(word2),
            '-2:word.lower()': word2.lower(),
            '-2:word.tag()': tagword2,
            #'-2:word.unaccent': remove_accents(word2),
            #'-2:word.middle_start': word2[:int(len_word2)],
            #'-2:word.middle_end': word2[int(len_word2):],
            #'-2:word.have_tone': True if len_tone2>0 else False,
            #'-2:word.len_tones': len_tone2,
            '-2:word.isdigit()': word2.isdigit(),
            '-2:word.ispunctuation': (word2 in string.punctuation)
        })

    if i > 2:
        word3 = sent[i-3][0]
        tagword3 = sent[i-3][1]
        len_tone3 = number_tone_word(word3)
        len_word3 = len(word3) / 2
        features.update({
            '-3:word': word3,
            '-3:word.position': i+3,
            '-3:word.letters': word_decomposition(word3),
            '-3:word.normalized': unicodedata.normalize('NFKD', word3),
            '-3:word.start_with_capital': word3[0].isupper(),
            '-3:len(word3)': len(word3),
            '-3:word.tag()': tagword3,
            #'-3:word.lower()': word2.lower(),
            #'-3:word.unaccent': remove_accents(word2),
            #'-3:word.middle_start': word2[:int(len_word2)],
            #'-3:word.middle_end': word2[int(len_word2):],
            #'-3:word.have_tone': True if len_tone2>0 else False,
            #'-3:word.len_tones': len_tone2,
            '-3:word.isdigit()': word3.isdigit(),
            '-3:word.ispunctuation': (word3 in string.punctuation)
        })
    """
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        tagword1 = sent[i+1][1]
        len_tone1 = number_tone_word(word1)
        len_word1 = len(word1) / 2
        features.update({
            '+1:word': word1,
            '+1:word.position': i+1,
            '+1:word.letters': word_decomposition(word1),
            '+1:word.normalized': unicodedata.normalize('NFKD', word1),
            '+1:word.start_with_capital': word1[0].isupper(),
            '+1:len(word2)': len(word1),
            '+1:word.lower()': word1.lower(),
            #'+1:word.tag()': tagword1,
            #'+1:word.unaccent': remove_accents(word1),
            #'+1:word.middle_start': word1[:int(len_word1)],
            #'+1:word.middle_end': word1[int(len_word1):],
            #'+1:word.have_tone': True if len_tone1>0 else False,
            #'+1:word.len_tones': len_tone1,
            '+1:word.isdigit()': word1.isdigit(),
            '+1:word.ispunctuation': (word1 in string.punctuation)
        })
    """ 
    if i < len(sent)-2:
        word2 = sent[i+2][0]
        tagword2 = sent[i+2][1]
        len_tone2 = number_tone_word(word2)
        len_word2 = len(word2) / 2
        features.update({
            '+2:word': word2,
            '+2:word.position': i+2,
            '+2:word.letters': word_decomposition(word2),
            '+2:word.normalized': unicodedata.normalize('NFKD', word2),
            '+2:word.start_with_capital': word2[0].isupper(),
            '+2:len(word2)': len(word2),
            '+2:word.tag()': tagword2,
            #'+2:word.lower()': word2.lower(),
            #'+2:word.unaccent': remove_accents(word2),
            #'+2:word.middle_start': word2[:int(len_word2)],
            #'+2:word.middle_end': word2[int(len_word2):],
            #'+2:word.have_tone': True if len_tone2>0 else False,
            #'+2:word.len_tones': len_tone2,
            '+2:word.isdigit()': word2.isdigit(),
            '+2:word.ispunctuation': (word2 in string.punctuation)
        })
        
    if i < len(sent)-3:
        word3 = sent[i+3][0]
        tagword3 = sent[i+3][1]
        len_tone3 = number_tone_word(word3)
        len_word3 = len(word3) / 2
        features.update({
            '+3:word': word3,
            '+3:word.position': i+3,
            '+3:word.letters': word_decomposition(word3),
            '+3:word.normalized': unicodedata.normalize('NFKD', word3),
            '+3:word.start_with_capital': word3[0].isupper(),
            '+3:len(word3)': len(word3),
            '+3:word.tag()': tagword3,
            #'+2:word.lower()': word2.lower(),
            #'+2:word.unaccent': remove_accents(word2),
            #'+2:word.middle_start': word2[:int(len_word2)],
            #'+2:word.middle_end': word2[int(len_word2):],
            #'+2:word.have_tone': True if len_tone2>0 else False,
            #'+2:word.len_tones': len_tone2,
            '+3:word.isdigit()': word3.isdigit(),
            '+3:word.ispunctuation': (word3 in string.punctuation)
        })
    """
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [word[1] for word in sent]

def sent2tokens(sent):
    return [word[0] for word in sent]

def format_data(csv_data):
    sents = []
    """for i in range(len(csv_data)):
        if math.isnan(float(csv_data.iloc[i, 0])): continue
        elif csv_data.iloc[i, 0] == 1.0:
            sents.append([[csv_data.iloc[i, 1], csv_data.iloc[i, 2]]])
        else:
            try: sents[-1].append([csv_data.iloc[i, 1], csv_data.iloc[i, 2]])
            except: print('...', csv_data.iloc[i, 2])
    for sent in sents:
        for i, word in enumerate(sent):
            if type(word[0]) != str:
                del sent[i]"""
    return csv_data
    

In [23]:
print(len(pd_augment_ner_iob_data), len(pd_test_data))

#train_sents = format_data(pd_train_data)
#test_sents = format_data(pd_test_data)
#dev_sents = format_data(pd_dev_data)
train_sents = [[word for word in sentence] for sentence in augmented_train_ner_data]
test_sents = [[word for word in sentence] for sentence in extracted_test_data]
dev_sents = [[word for word in sentence] for sentence in extracted_dev_data]

Xtrain = [sent2features(s) for s in train_sents]
ytrain = [sent2labels(s) for s in train_sents]

Xtest = [sent2features(s) for s in test_sents]
ytest = [sent2labels(s) for s in test_sents]

Xdev = [sent2features(s) for s in dev_sents]
ydev = [sent2labels(s) for s in dev_sents]



20391461 15924


In [ ]:
Xtrain[2]

# Modelling

In [14]:
#import pycrfsuite
import sklearn_crfsuite
import math, string, re
import scipy
import joblib
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from itertools import chain
from sklearn.preprocessing import MultiLabelBinarizer

In [26]:
import pycrfsuite


In [29]:

trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(Xtrain, ytrain):
    trainer.append(xseq, yseq)

In [40]:
project = "sangkak-02-2023-aug"
build_date = str(datetime.now()).replace(' ','_')
model_name = Path(f"crf_{project}_{build_date}.model")
model_file = str(model_name)
file_crf = Path(f"crf_{build_date}.object")

print(trainer.params())
trainer.set_params({
    #"algorithm": 'lbfgs',
    "c1": 0.0920512484757745,
    "c2": 0.0328771171605105, 
    "max_iterations":100,
    #"verbose":True,
    "num_memories":10000,
    "epsilon": 1e-3,
    "linesearch": "MoreThuente",
    "max_linesearch":100000,
    "delta":1e-4,
    #"n_job":-1,
    #"c": 2,
    #"pa_type": 2,
    "feature.possible_transitions":True,
    "feature.possible_states":True, 
    #"model_filename": model_file
})

['feature.minfreq', 'feature.possible_states', 'feature.possible_transitions', 'c1', 'c2', 'max_iterations', 'num_memories', 'epsilon', 'period', 'delta', 'linesearch', 'max_linesearch']


In [41]:
help(trainer.train)

Help on built-in function train:

train(...) method of pycrfsuite._pycrfsuite.Trainer instance
    BaseTrainer.train(self, model, int holdout=-1)
    
    Run the training algorithm.
    This function starts the training algorithm with the data set given
    by :meth:`Trainer.append` method.
    
    Parameters
    ----------
    model : string
        The filename to which the trained model is stored.
        If this value is empty, this function does not
        write out a model file.
    
    holdout : int, optional
        The group number of holdout evaluation. The
        instances with this group number will not be used
        for training, but for holdout evaluation.
        Default value is -1, meaning "use all instances for training".



In [ ]:
trainer.train('conll2002-esp.crfsuite')

In [25]:
project = "sangkak-02-2023-aug"
build_date = str(datetime.now()).replace(' ','_')
model_name = Path(f"crf_{project}_{build_date}.model")
model_file = str(model_name)
file_crf = Path(f"crf_{build_date}.object")

params = {
    "algorithm": 'lbfgs',
    "c1": 0.0920512484757745,
    "c2": 0.0328771171605105, 
    "max_iterations":100,
    "verbose":True,
    "num_memories":10000,
    "epsilon": 1e-3,
    "linesearch": "MoreThuente",
    "max_linesearch":100000,
    "delta":1e-4,
    #n_job=-1,
    #"c": 2,
    #"pa_type": 2,
    "all_possible_states":True,
    "all_possible_transitions":True, 
    "model_filename": model_file
}
crf = sklearn_crfsuite.CRF(**params)

crf.fit(Xtrain, ytrain, Xdev, ydev)    

final = {"crf": crf, "params": params}
joblib.dump(final, file_crf) 


loading training data to CRFsuite:  37%|███▋      | 447114/1219850 [02:56<05:04, 2536.05it/s]


KeyboardInterrupt: ignored

In [ ]:
# obtaining metrics such as accuracy, etc. on the train set
#crf_model = joblib.load(model_name)
#crf, params = crf_model['crf'], crf_model['params'] 
labels = list(crf.classes_)
labels.remove('O')

ypred = crf.predict(Xtrain)
print('- F1 score on the train set = {}'.format(metrics.flat_f1_score(ytrain, ypred, average='weighted', labels=labels, zero_division=False)))
print('- Accuracy on the train set = {}'.format(metrics.flat_accuracy_score(ytrain, ypred)))

sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
print(sorted_labels)
#print('Train set classification report: \n\n{}'.format(metrics.flat_classification_report(ytrain, 
#ypred, labels=sorted_labels, digits=3)))
#obtaining metrics such as accuracy, etc. on the test set
ypred = crf.predict(Xtest)
print('- F1 score on the test set = {}'.format(metrics.flat_f1_score(ytest, ypred, average='weighted', 
                                                labels=labels, zero_division=False)))
print('- Accuracy on the test set = {}'.format(metrics.flat_accuracy_score(ytest, ypred)))
print('Train set classification report: \n\n{}'.format(metrics.flat_classification_report(ytest, 
                            ypred, labels=sorted_labels, digits=3)))
sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))



In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
#print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-40:])

# Grid search

In [ ]:
scipy.stats.expon(scale=0.1)

In [ ]:
params = {
    "algorithm": 'lbfgs',
    "max_iterations":100,
    "verbose": False,
    #"job":-1,
    "all_possible_states":True,
    "all_possible_transitions":True, 
    "model_filename":model_file
}
crf_grill = sklearn_crfsuite.CRF(**params)

params_space = {
    'c1': scipy.stats.expon(scale=0.1),
    'c2': scipy.stats.expon(scale=0.05)
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf_grill, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=5,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(Xtrain, ytrain)

In [ ]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
#print(rs.cv_results_)
_x = [s['c1'] for s in rs.cv_results_['params']]
_y = [s['c2'] for s in rs.cv_results_['params']]
_c = [s for s in rs.cv_results_['mean_score_time']]

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(Xtest)
print(metrics.flat_classification_report(
    ytest, y_pred, labels=sorted_labels, digits=3
))

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])